In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
%cd /content/gdrive/My Drive/CV Project

Mounted at /content/gdrive/
/content/gdrive/My Drive/CV Project


In [2]:
import os, csv, torch, numpy as np, scipy.io, PIL.Image, torchvision.transforms, glob, matplotlib.pyplot as plt, cv2


In [3]:
def readImage(img_name):
    pil_image = PIL.Image.open(img_name).convert('RGB')
    img_original = np.array(pil_image)
    return img_original

def getColoredImage(img, new_color):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv_img)
    hsv_color = cv2.cvtColor(np.uint8([[new_color]]), cv2.COLOR_RGB2HSV)
    h.fill(hsv_color[0][0][0])
    s.fill(hsv_color[0][0][1])
    new_hsv_img = cv2.merge([h, s, v])
    new_rgb_img = cv2.cvtColor(new_hsv_img, cv2.COLOR_HSV2RGB)
    return new_rgb_img

def mergeImages(img, colored_img, wall):
    colored_img = cv2.bitwise_and(colored_img, colored_img, mask=wall)
    marked_img = cv2.bitwise_and(img, img, mask=cv2.bitwise_not(wall))
    final_img = cv2.bitwise_xor(colored_img, marked_img)
    return final_img

def saveImage(img_name, img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img_name= img_name[8:]
    cv2.imwrite("./groundtruth/" + img_name, img)

def changeColor(img_name, mask, new_color):
    img_original = readImage(img_name)
    colored_img = getColoredImage(img_original, new_color)
    final_img = mergeImages(img_original, colored_img, mask)
    saveImage(img_name, final_img)
    return final_img

def calculate_iou(mask1, mask2):
    # print(mask1)
    # Convert masks to binary imag
    mask1 = (mask1 > 0).astype(np.uint8)
    mask2 = (mask2 > 0).astype(np.uint8)

    # Calculate intersection and union
    intersection = np.sum(np.logical_and(mask1, mask2))
    union = np.sum(np.logical_or(mask1, mask2))

    # Calculate IoU
    iou = intersection / union if union > 0 else 0
    return iou



def resize_image(image, target_size):
    # Resize the image to the target size
    resized_image = cv2.resize(image, target_size)
    return resized_image


In [4]:
new_color = [111, 209, 201]
iou = 0
for i in range(1,101):
    m = 'bedroom/' + str(i) + 'wall*'
    masks = glob.glob(m)
    j = 'resnetcv/' + str(i) + '_img*'
    k = 'bedroom/' + str(i) + '_img*'
    img1 = glob.glob(j)
    final_img = readImage(img1[0])
    img2 = glob.glob(k)
    masks = [cv2.imread(mask) for mask in masks]
    masks = [cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY) for mask in masks]
    final_mask = masks[0]
    if len(masks) > 1:
        for mask in masks[1:]:
            final_mask = cv2.bitwise_or(final_mask, mask)

    true_img = changeColor(img2[0], final_mask, new_color)
    iou += calculate_iou(final_img, true_img)
    if i % 20 == 0:
        print('iteration: ', i)

iou = iou / 100
print('IoU: ', iou)

iteration:  20
iteration:  40
iteration:  60
iteration:  80
iteration:  100
IoU:  0.9878068225979227


In [5]:
new_color = [111, 209, 201]
iou = 0
for i in range(1,101):
    m = 'bedroom/' + str(i) + 'wall*'
    masks = glob.glob(m)
    j = 'hd_output/' + str(i) + '_img*'
    k = 'bedroom/' + str(i) + '_img*'
    img1 = glob.glob(j)
    final_img = readImage(img1[0])
    img2 = glob.glob(k)
    masks = [cv2.imread(mask) for mask in masks]
    masks = [cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY) for mask in masks]
    final_mask = masks[0]
    if len(masks) > 1:
        for mask in masks[1:]:
            final_mask = cv2.bitwise_or(final_mask, mask)

    true_img = changeColor(img2[0], final_mask, new_color)
    x,y,z = true_img.shape
    final_img = resize_image(final_img, (x,y))
    final_img = final_img.transpose((1,0,2))
    iou += calculate_iou(final_img, true_img)
    if i % 20 == 0:
        print('iteration: ', i)

iou = iou / 100
print('IoU: ', iou)

iteration:  20
iteration:  40
iteration:  60
iteration:  80
iteration:  100
IoU:  0.9846069768284345
